In [50]:
import brickschema
import requests
import json
import duckdb
import pandas as pd

In [51]:
con = duckdb.connect(database='data.duckdb', read_only=False)

In [52]:
con.execute("CREATE TABLE IF NOT EXISTS data (time TIMESTAMP, value REAL, id TEXT);")
# con.execute("CREATE INDEX IF NOT EXISTS data_id_idx on data(id);")
con.execute("CREATE UNIQUE INDEX IF NOT EXISTS data_value_unique_idx on data(time, id);")

In [53]:
g = brickschema.Graph()
g.load_file("ttl/Brick.ttl",)
g.load_file("ttl/building-compiled.ttl")

<Graph identifier=N558754c3be10403b9dc3b05ff5ca7357 (<class 'brickschema.graph.Graph'>)>

In [59]:
for point, data_id in g.query("""SELECT ?point ?data_id WHERE {
   ?sensor rdf:type/rdfs:subClassOf* brick:Point .
   ?sensor ref:hasExternalReference ?ref .
   ?ref a ref:TimeseriesReference .
   ?ref ref:hasTimeseriesId ?data_id 
}"""):
    body = {
        'pset': json.dumps({"points": [{'global': str(data_id)}]}),
        'num': 24 * 7
    }
    r = requests.post('https://alice.openmetrics.eu/historical/list/by/hour', data=body)
    ts = r.json().get('points', [])
    if len(ts) == 0 or len(ts[0]['measureList']) == 0:
        continue
    df = pd.DataFrame.from_records(ts[0]['timeseries'])
    df['id'] = data_id
    con.executemany('INSERT INTO data(time, value, id) VALUES (?, ?, ?)', df[['timestamp', 'value', 'id']].values)
    print(f"Wrote {len(df)} values for id {data_id}")

Wrote 642 values for id TUC.245.77.R4
Wrote 594 values for id TUC.245.77.R6
Wrote 596 values for id TUC.245.77.R2
Wrote 594 values for id TUC.245.77.R8
Wrote 594 values for id TUC.245.77.R9
Wrote 1451 values for id TUC.245.77.R0
Wrote 611 values for id TUC.245.77.R104
Wrote 593 values for id TUC.245.77.R106
Wrote 656 values for id TUC.245.77.R102
Wrote 593 values for id TUC.245.77.R108
Wrote 593 values for id TUC.245.77.R109
Wrote 1757 values for id TUC.245.77.R100
Wrote 633 values for id TUC.245.77.R84
Wrote 594 values for id TUC.245.77.R86
Wrote 599 values for id TUC.245.77.R82
Wrote 593 values for id TUC.245.77.R88
Wrote 593 values for id TUC.245.77.R89
Wrote 1255 values for id TUC.245.77.R80
Wrote 630 values for id TUC.245.77.R64
Wrote 594 values for id TUC.245.77.R66
Wrote 625 values for id TUC.245.77.R62
Wrote 594 values for id TUC.245.77.R68
Wrote 594 values for id TUC.245.77.R69
Wrote 1600 values for id TUC.245.77.R60
Wrote 594 values for id TUC.245.77.R44
Wrote 594 values for 

In [ ]:
con.close()